In [1]:
import geodb.model
from geodb.model import GPSPoint, db_url, GPSTrack

In [2]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [3]:
engine = sqlalchemy.create_engine(db_url(), echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
import pandas as pd
import numpy as np

Find longest tracks

In [10]:
query = """
select track.*, count(*), min(time) as start, max(time) as end, max(time) - min(time) as elapsed
from track
join point on track_id = track.id
group by track.id
order by elapsed desc
"""
display(pd.read_sql_query(query, engine).dropna(subset=['elapsed']))

,id,name,comment,description,source,type,filename,properties,raw,count,start,end,elapsed
8,2503,Aneel Nazareth (9962922),None,Aneel Nazareth (9962922),None,None,inreach.gpx,{},None,23,2019-12-09 22:43:30+00:00,2019-12-11 02:46:30+00:00,1 days 04:03:00
9,2500,Aneel Nazareth (9953683),None,Aneel Nazareth (9953683),None,None,inreach.gpx,{},None,2048,2019-12-07 00:03:15+00:00,2019-12-07 23:40:29+00:00,0 days 23:37:14
10,2497,Aneel Nazareth (9947046),None,Aneel Nazareth (9947046),None,None,inreach.gpx,{},None,2111,2019-12-04 23:35:00+00:00,2019-12-05 21:53:17+00:00,0 days 22:18:17
11,2502,Aneel Nazareth (9959973),None,Aneel Nazareth (9959973),None,None,inreach.gpx,{},None,545,2019-12-09 00:15:00+00:00,2019-12-09 21:23:59+00:00,0 days 21:08:59
12,2501,Aneel Nazareth (9956605),None,Aneel Nazareth (9956605),None,None,inreach.gpx,{},None,1785,2019-12-07 23:40:45+00:00,2019-12-08 14:10:00+00:00,0 days 14:29:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,1543,None,None,None,None,car,2019-12-02.gpx,{},None,1,2019-12-02 01:47:18+00:00,2019-12-02 01:47:18+00:00,0 days 00:00:00
2502,1232,None,None,None,None,car,2019-11-30 2.gpx,{},None,1,2019-11-30 03:50:46+00:00,2019-11-30 03:50:46+00:00,0 days 00:00:00
2503,40,Austin Bergstrom International Airport (AUS),None,None,None,Waypoint,None,None,None,1,2019-11-23 09:56:12+00:00,2019-11-23 09:56:12+00:00,0 days 00:00:00
2504,1122,Kii-Katsuura Station (紀伊勝浦駅),None,None,None,Waypoint,None,None,None,1,2019-11-29 07:48:13+00:00,2019-11-29 07:48:13+00:00,0 days 00:00:00


Pick a track

In [11]:
track = session.query(GPSTrack).get(2503)

In [12]:
track.name

'Aneel Nazareth (9962922)'

Find all overlapping tracks

In [16]:
query = """
select * from track where id in
(select distinct(track_id) from point
where point.time between (select min(time) from point where track_id = 2503) 
                     and (select max(time) from point where track_id = 2503))
"""
display(pd.read_sql_query(query, engine))

,id,name,comment,description,source,type,filename,properties,raw
0,2375,None,None,None,None,walking,2019-12-09.gpx,{},None
1,2376,None,None,None,None,stationary,2019-12-09.gpx,{},None
2,2377,None,None,None,None,walking,2019-12-09.gpx,{},None
3,2378,None,None,None,None,stationary,2019-12-09.gpx,{},None
4,2379,None,None,None,None,train,2019-12-09.gpx,{},None
...,...,...,...,...,...,...,...,...,...
102,2503,Aneel Nazareth (9962922),None,Aneel Nazareth (9962922),None,None,inreach.gpx,{},None
103,2504,AAL1584-20191210,None,KAUS - KDFW,None,None,FlightAware_AAL1584_KAUS_KDFW_20191210.gpx,{},None
104,2505,AAL1584-20191210,None,KAUS - KDFW,None,None,FlightAware_AAL1584_KAUS_KDFW_20191210.gpx,{},None
105,2508,AAL176-20191210,None,RJAA - KDFW,None,None,FlightAware_AAL176_RJAA_KDFW_20191210.gpx,{},None


In [17]:
overlapping = [session.query(GPSTrack).get(int(t)) for t in pd.read_sql_query(query, engine)[['id']].values]

In [18]:
len(overlapping)

107

In [19]:
query = """
select track.*, count(*), min(time) as start, max(time) as end, max(time) - min(time) as elapsed
from track
join point on track.id = point.track_id
where point.time between (select min(p2.time) from point p2 where track_id = 2503)
                     and (select max(p3.time) from point p3 where track_id = 2503)
group by track.id
order by elapsed desc
"""
display(pd.read_sql_query(query, engine))

,id,name,comment,description,source,type,filename,properties,raw,count,start,end,elapsed
0,2503,Aneel Nazareth (9962922),None,Aneel Nazareth (9962922),None,None,inreach.gpx,{},None,23,2019-12-09 22:43:30+00:00,2019-12-11 02:46:30+00:00,1 days 04:03:00
1,2425,None,None,None,None,airplane,2019-12-09.gpx,{},None,53,2019-12-10 02:29:00+00:00,2019-12-10 13:11:07+00:00,0 days 10:42:07
2,2449,None,None,None,None,airplane,2019-12-10.gpx,{},None,53,2019-12-10 02:29:00+00:00,2019-12-10 13:11:07+00:00,0 days 10:42:07
3,2509,20191210,None,None,None,None,FlightAware_AAL176_RJAA_KDFW_20191210.gpx,{},None,400,2019-12-10 06:01:17+00:00,2019-12-10 13:07:36+00:00,0 days 07:06:19
4,2508,AAL176-20191210,None,RJAA - KDFW,None,None,FlightAware_AAL176_RJAA_KDFW_20191210.gpx,{},None,111,2019-12-10 02:37:43+00:00,2019-12-10 05:59:52+00:00,0 days 03:22:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,2472,None,None,None,None,stationary,2019-12-10.gpx,{},None,1,2019-12-10 14:29:38+00:00,2019-12-10 14:29:38+00:00,0 days 00:00:00
103,2389,None,None,None,None,stationary,2019-12-09.gpx,{},None,1,2019-12-10 00:17:22+00:00,2019-12-10 00:17:22+00:00,0 days 00:00:00
104,2388,None,None,None,None,airplane,2019-12-09.gpx,{},None,1,2019-12-10 00:17:14+00:00,2019-12-10 00:17:14+00:00,0 days 00:00:00
105,2387,None,None,None,None,stationary,2019-12-09.gpx,{},None,1,2019-12-10 00:16:26+00:00,2019-12-10 00:16:26+00:00,0 days 00:00:00


In [41]:
from geojson import FeatureCollection

In [81]:
def feature_with_on_click(track):
    f = track.as_geojson_feature(True)
    f.popup = f"{track.id}: {track.name}"
    return f

In [82]:
features = FeatureCollection([feature_with_on_click(t) for t in overlapping])

import json
from decimal import Decimal

def default(obj):
    if isinstance(obj, Decimal):
        return str(obj)
    raise TypeError("Object of type '%s' is not JSON serializable" % type(obj).__name__)

geojson = json.dumps(features, default=default)


In [43]:
from IPython.display import GeoJSON

In [83]:
layers = GeoJSON(features)

In [86]:
print(layers)

<IPython.core.display.GeoJSON object>


In [77]:
clicked = []
def my_method(**kwargs):
    #do your code here
    global clicked
    clicked.add(kwargs)

layers.on_click(my_method)

<function my_method at 0x7f22a5130b70>


In [84]:
layers

<IPython.display.GeoJSON object>

In [80]:
clicked

[]

In [46]:
def clone_model(model, **kwargs):
    """Clone an arbitrary sqlalchemy model object without its primary key values."""
    # Ensure the model’s data is loaded before copying.
    model.id

    table = model.__table__
    non_pk_columns = [k for k in table.columns.keys() if k not in table.primary_key]
    data = {c: getattr(model, c) for c in non_pk_columns}
    data.update(kwargs)

    clone = model.__class__(**data)
    return clone

In [53]:
def overlapping_points(session, track_id):
    track = session.query(GPSTrack).get(track_id)
    points = (
        session.query(GPSPoint)
            .filter(GPSPoint.time.between(track.points[0].time, track.points[-1].time))
            .order_by(GPSPoint.time)
    )
    overlap = GPSTrack()
    session.add(overlap)
    overlap.points = [clone_model(p, track_id=track_id) for p in points]
    return overlap

In [54]:
overlap = overlapping_points(session, 2503)

In [55]:
overlap

In [56]:
merged = FeatureCollection([overlap.as_geojson_feature(True)])

In [58]:
GeoJSON(merged)

<IPython.display.GeoJSON object>